In [1]:
import findspark
findspark.init()
import pyspark
import random
from decimal import Decimal
sc = pyspark.SparkContext()
import json

In [2]:
def unmarshal_dynamodb_json(node):
    data = dict({})
    data['M'] = node
    return _unmarshal_value(data)


def _unmarshal_value(node):
    if type(node) is not dict:
        return node

    for key, value in node.items():
        # S – String - return string
        # N – Number - return int or float (if includes '.')
        # B – Binary - not handled
        # BOOL – Boolean - return Bool
        # NULL – Null - return None
        # M – Map - return a dict
        # L – List - return a list
        # SS – String Set - not handled
        # NN – Number Set - not handled
        # BB – Binary Set - not handled
        key = key.lower()
        if key == 'bool':
            return value
        if key == 'null':
            return None
        if key == 's':
            return value
        if key == 'n':
            if '.' in str(value):
                return Decimal(value)
            return int(value)
        if key in ['m', 'l']:
            if key == 'm':
                data = {}
                for key1, value1 in value.items():
                    if key1.lower() == 'l':
                        data = [_unmarshal_value(n) for n in value1]
                    else:
                        if type(value1) is not dict:
                            return _unmarshal_value(value)
                        data[key1] = _unmarshal_value(value1)
                return data
            data = []
            for item in value:
                data.append(_unmarshal_value(item))
            return data

In [3]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
log_txt=sc.textFile("parsing.txt")

In [7]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

dfout = log_txt.map(lambda j: unmarshal_dynamodb_json(json.loads(j)))
# sqlContext.udf.register('unmarshal_dynamodb_json', )
# dfout = udf(unmarshal_dynamodb_json, StructType())

log_txt1=sqlContext.read.text("parsing.txt")
df = sqlContext.read.json(log_txt1.rdd.map(lambda j: json.dumps(unmarshal_dynamodb_json(json.loads(j['value'])))))

In [8]:
print(dfout.first())

{u'updated': u'2017-09-21T20:00:53.065618', u'is_fake_company': False, u'tenant_id': 1, u'is_shiftgig_company': False, u'is_status_company': True, u'company_id': 251, u'company_name': u'Sodexo | Northwestern University:McCormick School of Engineering', u'is_geo_enabled_company': False, u'company_market_id': 1, u'is_dead_company': False}


In [9]:
print(df.first())

Row(company_id=251, company_market_id=1, company_name=u'Sodexo | Northwestern University:McCormick School of Engineering', is_dead_company=False, is_fake_company=False, is_geo_enabled_company=False, is_shiftgig_company=False, is_status_company=True, tenant_id=1, updated=u'2017-09-21T20:00:53.065618')


In [10]:
type(dfout)

pyspark.rdd.PipelinedRDD

In [11]:
type(df)

pyspark.sql.dataframe.DataFrame

In [12]:
dfout.take(5)

[{u'company_id': 251,
  u'company_market_id': 1,
  u'company_name': u'Sodexo | Northwestern University:McCormick School of Engineering',
  u'is_dead_company': False,
  u'is_fake_company': False,
  u'is_geo_enabled_company': False,
  u'is_shiftgig_company': False,
  u'is_status_company': True,
  u'tenant_id': 1,
  u'updated': u'2017-09-21T20:00:53.065618'},
 {u'company_id': 3757,
  u'company_market_id': 10,
  u'company_name': u'The Victory Ship Inc.',
  u'is_dead_company': False,
  u'is_fake_company': False,
  u'is_geo_enabled_company': False,
  u'is_shiftgig_company': False,
  u'is_status_company': True,
  u'tenant_id': 1,
  u'updated': u'2018-04-13T09:38:34.977732'},
 {u'company_id': 32,
  u'company_market_id': 1,
  u'company_name': u'Do Not Use- Levy Restaurants',
  u'is_dead_company': True,
  u'is_fake_company': False,
  u'is_geo_enabled_company': False,
  u'is_shiftgig_company': False,
  u'is_status_company': True,
  u'tenant_id': 1,
  u'updated': u'2017-09-21T20:00:53.065618'},
 {

In [13]:
df.take(5)

[Row(company_id=251, company_market_id=1, company_name=u'Sodexo | Northwestern University:McCormick School of Engineering', is_dead_company=False, is_fake_company=False, is_geo_enabled_company=False, is_shiftgig_company=False, is_status_company=True, tenant_id=1, updated=u'2017-09-21T20:00:53.065618'),
 Row(company_id=3757, company_market_id=10, company_name=u'The Victory Ship Inc.', is_dead_company=False, is_fake_company=False, is_geo_enabled_company=False, is_shiftgig_company=False, is_status_company=True, tenant_id=1, updated=u'2018-04-13T09:38:34.977732'),
 Row(company_id=32, company_market_id=1, company_name=u'Do Not Use- Levy Restaurants', is_dead_company=True, is_fake_company=False, is_geo_enabled_company=False, is_shiftgig_company=False, is_status_company=True, tenant_id=1, updated=u'2017-09-21T20:00:53.065618'),
 Row(company_id=3517, company_market_id=28, company_name=u'Initech NY', is_dead_company=False, is_fake_company=True, is_geo_enabled_company=False, is_shiftgig_company=

In [14]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
df_csv = sqlContext.createDataFrame(dfout, ['company_id', 'company_market_id','company_name','is_dead_company','is_fake_company','is_geo_enabled_company','is_shiftgig_company','is_status_company','tenant_id','updated'])

/usr/local/opt/apache-spark/libexec/python/pyspark/sql/session.py:360: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


In [15]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
df1_csv = sqlContext.createDataFrame(dfout, ['company_id', 'company_market_id','company_name','is_dead_company','is_fake_company','is_geo_enabled_company','is_shiftgig_company','is_status_company','tenant_id','updated'])

In [18]:
df_csv.coalesce(1).write.format('com.databricks.spark.csv').options(header='true').save('csv_out1.csv')


In [17]:
df1_csv.coalesce(1).write.format('com.databricks.spark.csv').options(header='true').save('csv1_out.csv')


## AWS Glue script

In [ ]:
import sys
import json
from decimal import Decimal
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

## @params: [JOB_NAME]
args = getResolvedOptions(sys.argv, ['JOB_NAME'])

sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
job.init(args['JOB_NAME'], args)


def unmarshal_dynamodb_json(node):
    data = dict({})
    data['M'] = node
    return _unmarshal_value(data)


def _unmarshal_value(node):
    if type(node) is not dict:
        return node

    for key, value in node.items():
        # S – String - return string
        # N – Number - return int or float (if includes '.')
        # B – Binary - not handled
        # BOOL – Boolean - return Bool
        # NULL – Null - return None
        # M – Map - return a dict
        # L – List - return a list
        # SS – String Set - not handled
        # NN – Number Set - not handled
        # BB – Binary Set - not handled
        key = key.lower()
        if key == 'bool':
            return value
        if key == 'null':
            return None
        if key == 's':
            return value
        if key == 'n':
            if '.' in str(value):
                return Decimal(value)
            return int(value)
        if key in ['m', 'l']:
            if key == 'm':
                data = {}
                for key1, value1 in value.items():
                    if key1.lower() == 'l':
                        data = [_unmarshal_value(n) for n in value1]
                    else:
                        if type(value1) is not dict:
                            return _unmarshal_value(value)
                        data[key1] = _unmarshal_value(value1)
                return data
            data = []
            for item in value:
                data.append(_unmarshal_value(item))
            return data



connection_options = {"paths": [ "s3://prod-starbase-companybase/2018-04-27-19-28-23/e82ca4a3-6d76-4f9c-bfbd-aaad27cbf161"]}
input_dyf = glueContext.create_dynamic_frame.from_options(connection_type='s3', connection_options=connection_options)
company_base = input_dyf.toDF()
company_final = sqlContext.read.json(company_base.rdd.map(lambda r : json.dumps(unmarshal_dynamodb_json(json.loads(r['value'])))))
company_dynamic = DynamicFrame.fromDF(company_final, glueContext, "company_dynamic")

## @type: DropNullFields
## @args: [transformation_ctx = "dropnullfields3"]
## @return: dropnullfields3
## @inputs: [frame = resolvechoice2]
dropnullfields3 = DropNullFields.apply(frame = company_dynamic, transformation_ctx = "dropnullfields3")
## @type: DataSink
## @args: [connection_type = "s3", connection_options = {"path": "s3://shiftgig-data-us-east-1-warehouse/orc/company_base/"}, format = "orc", transformation_ctx = "datasink4"]
## @return: datasink4
## @inputs: [frame = dropnullfields3]
datasink4 = glueContext.write_dynamic_frame.from_options(frame = dropnullfields3, connection_type = "s3", connection_options = {"path": "s3://shiftgig-data-us-east-1-warehouse/orc/company_base/"}, format = "orc", transformation_ctx = "datasink4")
job.commit()

In [ ]:
import sys
import json
from decimal import Decimal
from pyspark.sql import SQLContext
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.dynamicframe import DynamicFrame

## @params: [JOB_NAME]
#will inherit arguments from the lambda we have yet to make#
args = getResolvedOptions(sys.argv,
                          ['JOB_NAME',
                           's3_object_key',
                           'dataset_name',
                           ])


sc = SparkContext()
glueContext = GlueContext(sc)
sqlContext = SQLContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
job.init(args['JOB_NAME'], args)


def unmarshal_dynamodb_json(node):
    data = dict({})
    data['M'] = node
    return _unmarshal_value(data)


def _unmarshal_value(node):
    if type(node) is not dict:
        return node

    for key, value in node.items():
        # S – String - return string
        # N – Number - return int or float (if includes '.')
        # B – Binary - not handled
        # BOOL – Boolean - return Bool
        # NULL – Null - return None
        # M – Map - return a dict
        # L – List - return a list
        # SS – String Set - not handled
        # NN – Number Set - not handled
        # BB – Binary Set - not handled
        key = key.lower()
        if key == 'bool':
            return value
        if key == 'null':
            return None
        if key == 's':
            return value
        if key == 'n':
            if '.' in str(value):
                return Decimal(value)
            return int(value)
        if key in ['m', 'l']:
            if key == 'm':
                data = {}
                for key1, value1 in value.items():
                    if key1.lower() == 'l':
                        data = [_unmarshal_value(n) for n in value1]
                    else:
                        if type(value1) is not dict:
                            return _unmarshal_value(value)
                        data[key1] = _unmarshal_value(value1)
                return data
            data = []
            for item in value:
                data.append(_unmarshal_value(item))
            return data


dataset = sqlContext.read.text(args['s3_object_key'])
dataset_final = sqlContext.read.json(dataset.rdd.map(lambda r : json.dumps(unmarshal_dynamodb_json(json.loads(r['value'])))))
dataset_dynamic = DynamicFrame.fromDF(dataset_final, glueContext, "dataset_dynamic")

## @type: DropNullFields
## @args: [transformation_ctx = "dropnullfields3"]
## @return: dropnullfields3
## @inputs: [frame = resolvechoice2]
dropnullfields3 = DropNullFields.apply(frame = dataset_dynamic, transformation_ctx = "dropnullfields3")
## @type: DataSink
## @args: [connection_type = "s3", connection_options = {"path": "s3://shiftgig-data-us-east-1-warehouse/orc/company_base/"}, format = "orc", transformation_ctx = "datasink4"]
## @return: datasink4
## @inputs: [frame = dropnullfields3]
path = "s3://shiftgig-data-us-east-1-warehouse/orc/{}/".format(args['dataset_name'])
datasink4 = glueContext.write_dynamic_frame.from_options(frame = dropnullfields3, connection_type = "s3", connection_options = {"path": path}, format = "orc", transformation_ctx = "datasink4")
job.commit()